<a href="https://colab.research.google.com/github/dasd412/my-first-ai/blob/main/GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 훈련 세트 : 테스트 세트 (8:2)에서 훈련 세트 : 테스트 세트 : 검증 세트 (6:2:2)로 바꾼다.
# 1. 훈련 세트에서 모델 훈련 -> 검증 세트로 모델 평가
# 2. 매개 변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델 다시 훈련
# 3. 마지막에 테스트세트에서 최종 점수 평가

In [ ]:
import pandas as pd

wine=pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
wine.head()

In [ ]:
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42)

In [ ]:
# 훈련세트에서 다시 검증 세트로 나누기
sub_input,val_input,sub_target,val_target=train_test_split(train_input,train_target,test_size=0.2,random_state=42)

In [ ]:
print(sub_input.shape,val_input.shape)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)

print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))
# 훈련세트에 과대 적합되어 있으므로 매개 변수를 바꿔서 더 좋은 모델을 찾아야 함

In [ ]:
# 교차 검증 : 검증 세트를 뗴어서 평가하는 과정을 여러번 반복하는 것
from sklearn.model_selection import cross_validate

scores=cross_validate(dt,train_input,train_target)
print(scores)

In [ ]:
# 교차 검증의 최종 점수 얻는 법
import numpy as np

print(np.mean(scores['test_score']))

In [ ]:
# 교차 검증 자체로는 훈련 세트를 섞어 폴드를 나누지 않는다.
# 그래서 섞어서 폴드를 나누려면 분할기를 지정해야 한다.
# cross_validate() 함수는 회귀모델인 경우 KFold 분할기를, 분류 모델인 경우 StratifiedKFold 분할기를 사용한다.

from sklearn.model_selection import StratifiedKFold
scores=cross_validate(dt,train_input,train_target,cv=StratifiedKFold())
print(np.mean(scores['test_score']))

In [ ]:
# n_splits 매개변수는 몇번 폴드 교차 검증을 할 지 정하는 것이다.
splitter=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores=cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))

In [ ]:
#하이퍼 파라미터 튜닝
#최적의 매개변수를 자동으로 찾아주는 도구 : 그리드 서치

from sklearn.model_selection import GridSearchCV

params={'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

In [ ]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)

In [ ]:
gs.fit(train_input,train_target)

In [ ]:
# 모델 중에서 검증 점수가 가정 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델 훈련
dt=gs.best_estimator_
print(dt.score(train_input,train_target))

In [ ]:
# 그리드 서치로 찾은 최적의 매개변수
print(gs.best_params_)

In [ ]:
# 각 매개 변수에서 수행한 교차 검증의 평균 점수
print(gs.cv_results_['mean_test_score'])

In [ ]:
best_index=np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index]) # 최상의 검증 점수를 만든 매개 변수 조합

In [ ]:
params={
    'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
    'max_depth': range(5,20,1),
    'min_samples_split':range(2,100,10)
}

In [ ]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)

In [ ]:
print(gs.best_params_)

In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

In [ ]:
from scipy.stats import uniform,randint

In [ ]:
rgen=randint(0,10)
rgen.rvs(10)

In [ ]:
np.unique(rgen.rvs(1000),return_counts=True)

In [ ]:
ugen=uniform(0,1)
ugen.rvs(10)

In [ ]:
params={
    'min_impurity_decrease':uniform(0.0001,0.001),
    'max_depth':randint(20,50),
    'min_samples_split':randint(2,25),
    'min_samples_leaf':randint(1,25),
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gs=RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)

In [ ]:
# params에 정의된 매개변수 범위에서 총 100번을 샘플링하여 교차검증을 수행하고, 최적의 매개변수 조합을 찾음.
print(gs.best_params_)

In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

In [ ]:
dt=gs.best_estimator_
print(dt.score(test_input,test_target))